In [ ]:
# Import the necessary library for reading the file.
import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from dask.distributed import Client
import dask.array as da
#import h5netcdf
#import netCDF4

In [ ]:
## Module load 
import os
import geopandas as gpd
import pandas as pd
import numpy as np
from typing import Union, List, Tuple

## Function to subset any given geofabric upstream of list of outlets
def subset_upstream_basins_shapefile(
    input_basin_path: str,
    input_river_path: str,
    start_ranks: Union[int, float, List[int], np.ndarray, pd.Series],
    rank_id_col: str,
    next_id_col: str,
    unit_area_col: str,
    upstream_area_col: str,
    output_basin_path: str,
    output_river_path: str,
    include_downstream: bool = False
) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """
    Subset upstream basins from a GeoFabric based on outlet rank IDs.
    Optionally includes the immediate downstream basin(s) and updates their upstream area.

    Parameters:
        input_basin_path (str): Path to input basin shapefile.
        input_river_path (str): Path to input river shapefile.
        start_ranks: Outlet rank(s) to start upstream search (int, list, np.ndarray, or Series).
        rank_id_col (str): Column name for basin ID.
        next_id_col (str): Column name for downstream ID.
        unit_area_col (str): area of the subbasin.
        upstream_area_col (str): Column to update with total upstream area.
        output_basin_path (str): Path to save output basin shapefile.
        output_river_path (str): Path to save output river shapefile.
        include_downstream (bool): Include immediate downstream basin(s) and update area.

    Returns:
        Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]: Subsetted basin and river GeoDataFrames.
    """
    gdf_basin = gpd.read_file(input_basin_path).copy()
    gdf_river = gpd.read_file(input_river_path).copy()

    # Validate required columns
    required_cols = [rank_id_col, next_id_col, upstream_area_col, unit_area_col]
    missing_cols = [col for col in required_cols if col not in gdf_basin.columns]
    if missing_cols:
        raise ValueError(f"Missing required column(s) in basin shapefile: {', '.join(missing_cols)}")

    # Normalize start_ranks
    if isinstance(start_ranks, (int, float)):
        start_ranks = [int(start_ranks)]
    elif isinstance(start_ranks, (np.ndarray, pd.Series)):
        start_ranks = start_ranks.astype(int).tolist()
    elif isinstance(start_ranks, list):
        start_ranks = list(map(int, start_ranks))
    else:
        raise TypeError(f"Unsupported type for start_ranks: {type(start_ranks).__name__}")

    # Build maps
    downstream_map = gdf_basin.set_index(rank_id_col)[next_id_col].to_dict()
    upstream_map = (
        gdf_basin[[rank_id_col, next_id_col]]
        .dropna()
        .groupby(next_id_col)[rank_id_col]
        .apply(list)
        .to_dict()
    )

    # Determine true outlet basins
    start_set = set(start_ranks)
    nested_outlets = {r for r in start_set if downstream_map.get(r) in start_set}
    true_outlets = start_set - nested_outlets
    if not true_outlets:
        raise ValueError("No valid outlets found (possibly circular or nested relationships).")

    # Traverse upstream
    visited = set()
    stack = list(true_outlets)
    while stack:
        current = stack.pop()
        if current not in visited:
            visited.add(current)
            stack.extend(upstream_map.get(current, []))

    downstream_ids = []

    if include_downstream:
        # Add immediate downstream(s) if not already included
        downstream_ids = [
            downstream_map[o] for o in true_outlets
            if downstream_map.get(o) not in visited and downstream_map.get(o) is not None
        ]
        visited.update(downstream_ids)

        # Update next_id_col to mark outlet point
        gdf_basin.loc[gdf_basin[rank_id_col].isin(downstream_ids), next_id_col] = -9999

        # Update upstream area of downstream basins
        for outlet in true_outlets:
            downstream_id = downstream_map.get(outlet)
            if downstream_id in downstream_ids:
                try:
                    outlet_area = gdf_basin.loc[gdf_basin[rank_id_col] == outlet, upstream_area_col].values[0]
                    unit_area = gdf_basin.loc[gdf_basin[rank_id_col] == downstream_id, unit_area_col].values[0]
                    gdf_basin.loc[gdf_basin[rank_id_col] == downstream_id, upstream_area_col] = outlet_area + unit_area
                except IndexError:
                    continue
    else:
        gdf_basin.loc[gdf_basin[rank_id_col].isin(true_outlets), next_id_col] = -9999

    # Subset basin and river layers
    gdf_basin = gdf_basin[gdf_basin[rank_id_col].isin(visited)].copy()
    gdf_river = gdf_river[gdf_river[rank_id_col].isin(gdf_basin[rank_id_col])].copy()

    # Save to files
    os.makedirs(os.path.dirname(output_basin_path), exist_ok=True)
    gdf_basin.to_file(output_basin_path, driver='ESRI Shapefile')
    print(f"✅ Saved basin subset to: {output_basin_path}")

    os.makedirs(os.path.dirname(output_river_path), exist_ok=True)
    gdf_river.to_file(output_river_path, driver='ESRI Shapefile')
    print(f"✅ Saved river subset to: {output_river_path}")

    return gdf_basin, gdf_river
    

## Function to subset input netcdf file for the study of interest

def subset_netcdf_ddb(
    input_basin_path,
    input_ddb_nc_path, 
    output_ddb_nc_path 
):
    # Load basin shapefile
    gdf = gpd.read_file(input_basin_path)
    print(f"Loaded basin shapefile with {len(gdf)} features")

    # Open DDB and reorder
    with xr.open_dataset(input_ddb_nc_path, chunks={}) as ds_ddb:
        rank_id_ddb = ds_ddb['Rank'].dropna(dim='subbasin').values.astype(int)
        next_id_ddb = ds_ddb['Next'].dropna(dim='subbasin').values.astype(int)
        comid_id_ddb = ds_ddb['subbasin'].dropna(dim='subbasin').values.astype(int)

        if 'Rank' not in gdf.columns or 'Next' not in gdf.columns:
            df = pd.DataFrame({'COMID': comid_id_ddb, 'Rank': rank_id_ddb, 'Next': next_id_ddb}).dropna().astype(int)
            gdf = gdf.merge(df, on='COMID', how='left')

        if 'Next' not in gdf.columns:
            gdf['Next'] = np.nan

        gdf.loc[gdf['NextDownID'] == -9999.0, 'Next'] = 0
        gdf['Rank_int'] = gdf['Rank'].astype('Int64')
        gdf['Next_int'] = gdf['Next'].fillna(0).astype(int)

        outlet_ranks = gdf.loc[gdf['NextDownID'] == -9999.0, 'Rank_int'].dropna().tolist()
        non_outlet_ranks = gdf.loc[gdf['NextDownID'] != -9999.0, 'Rank_int'].dropna().tolist()
        reordered = np.concatenate([sorted(non_outlet_ranks), sorted(outlet_ranks)])
        reordered_list = list(reordered)

        rank_to_index_ddb = {r: i for i, r in enumerate(rank_id_ddb)}
        reordered_indices_ddb = [rank_to_index_ddb[r] for r in reordered if r in rank_to_index_ddb]

        subbasin_dim_ddb = 'subbasin' if 'subbasin' in ds_ddb.dims else list(ds_ddb.dims)[0]
        ds_ddb_subset = ds_ddb.isel({subbasin_dim_ddb: reordered_indices_ddb})

        subbasin_sub_ddb = ds_ddb_subset['subbasin'].values.astype(int) if 'subbasin' in ds_ddb_subset.coords else np.arange(len(reordered_indices_ddb))

        rank_map = {r: i + 1 for i, r in enumerate(reordered)}
        gdf_indexed = gdf.set_index('Rank_int')
        gdf_sub = gdf_indexed.loc[reordered]
        new_rank = np.array([rank_map[r] for r in reordered])
        new_next = np.array([rank_map.get(n, 0) for n in gdf_sub['Next_int'].values])

        ds_ddb_sub = ds_ddb_subset.assign_coords({subbasin_dim_ddb: np.arange(len(new_rank))})
        ds_ddb_sub = ds_ddb_sub.assign(
            Rank_old=(subbasin_dim_ddb, reordered),
            Next_old=(subbasin_dim_ddb, gdf_sub['Next_int'].values),
            subbasin=(subbasin_dim_ddb, subbasin_sub_ddb),
            Rank=(subbasin_dim_ddb, new_rank),
            Next=(subbasin_dim_ddb, new_next)
        ).sortby("Rank")

        if 'lat' in ds_ddb and 'lon' in ds_ddb:
            ds_ddb_sub = ds_ddb_sub.assign_coords(
                lat=ds_ddb['lat'].isel({subbasin_dim_ddb: reordered_indices_ddb}),
                lon=ds_ddb['lon'].isel({subbasin_dim_ddb: reordered_indices_ddb})
            )

        ds_ddb_sub.to_netcdf(output_ddb_nc_path)
        
        max_rank = len(ds_ddb_sub['Rank'])
        
        print(f"Saved DDB subset to {output_ddb_nc_path}")
        
    return ds_ddb_sub, max_rank


## Function to Subset the Parameters and Climate netcdf file

def subset_netcdf_climate_and_parameters(
    input_ddb_path,
    input_nc_path,
    output_nc_path,
    Key_column,
    lat_name,
    lon_name
):
    # Load datasets
    with xr.open_dataset(input_nc_path, chunks={}) as data_nc, xr.open_dataset(input_ddb_path) as mask_nc:
        # Step 1: Sort data_nc by ID
        data_ids = data_nc[Key_column].values
        sort_idx = np.argsort(data_ids)
        data_sorted = data_nc.isel(subbasin=sort_idx)
    
        # Step 2: Get Rank_old and convert to 0-based index
        rank = mask_nc["Rank_old"].values.astype(int)
        valid = rank > 0
        rank = rank[valid]
        mask_ids = rank - 1  # These are the positional indices in data_sorted

        # Step 3: Subset data using sorted Rank_old order
        subset = data_sorted.isel(subbasin=xr.DataArray(mask_ids, dims="subbasin"))

        # Step 4: Assign correct coordinates from mask file
        subset = subset.assign_coords({
            Key_column: subset[Key_column],
            lat_name: subset[lat_name],
            lon_name: subset[lon_name]
        })   
    
        # Step 5: Save
        subset.to_netcdf(output_nc_path)
    print(f"Processed and saved: {output_nc_path}")
    
    return subset

# Copy the required MESH input file
def edit_TotalNumOfGridsSubbasins_value(input_file_path, output_file_path, new_value):
        with open(input_file_path, 'r', encoding='utf-8') as src:
            lines = src.readlines()
            line = lines[3]
            # Preserve leading spaces/tabs
            leading_ws = line[:len(line) - len(line.lstrip())]
            # Split while preserving spacing logic
            tokens = line.strip().split()
            tokens[7] = str(new_value)
            # Rebuild the line with original indentation
            lines[3] = leading_ws + '   '.join(tokens) + '\n'
        with open(output_file_path, 'w', encoding='utf-8') as dest:
            dest.writelines(lines)

# Function to read time series data from a netcdf file for a given subbasin
import xarray as xr
import numpy as np
import pandas as pd

def extract_timeseries_by_subbasin(nc_path, subbasin_dim, variable_name, target_subbasin=None, target_latlon=None, save_csv=True):
    """
    Extract a time series for a specific subbasin by index or nearest lat/lon.

    Parameters:
        nc_path (str): Path to NetCDF file
        subbasin_dim (str): Name of the subbasin dimension in the NetCDF file
        variable_name (str): Name of the variable to extract
        target_subbasin (int): Index of the subbasin (0-based)
        target_latlon (tuple): (lon, lat) to find the nearest subbasin
        save_csv (bool): Whether to save the result as a CSV

    Returns:
        pd.DataFrame: DataFrame with time series for the selected subbasin
    """
    ds = xr.open_dataset(nc_path)

    # Check if subbasin dimension exists
    if subbasin_dim not in ds.dims:
        raise KeyError(f"Dimension '{subbasin_dim}' not found in the dataset.")

    if variable_name not in ds.variables:
        raise KeyError(f"Variable '{variable_name}' not found in the dataset.")

    if target_latlon is not None:
        if 'longitude' not in ds or 'latitude' not in ds:
            raise KeyError("Latitude/Longitude variables not found in dataset.")

        target_lon, target_lat = target_latlon
        lon_vals = ds['longitude'].values
        lat_vals = ds['latitude'].values

        # Flatten in case of multidimensional lat/lon
        distances = np.sqrt((lon_vals - target_lon) ** 2 + (lat_vals - target_lat) ** 2)
        if distances.ndim > 1:
            distances = distances.flatten()

        target_subbasin = int(np.argmin(distances))
        print(f"Nearest subbasin index to (lon={target_lon}, lat={target_lat}) is {target_subbasin}")

    elif target_subbasin is None:
        raise ValueError("You must provide either a subbasin index or a lat/lon location.")

    # Extract data
    data = ds[variable_name].isel({subbasin_dim: target_subbasin}).load()

    # Extract time dimension (assumes 'time' exists)
    time = ds.coords['time'].values

    # Construct DataFrame
    df = pd.DataFrame({
        'time': time,
        variable_name: data.values
    })

    if save_csv:
        csv_name = f"subbasin_{target_subbasin}_{variable_name}.csv"
        df.to_csv(csv_name, index=False)
        print(f"Saved to {csv_name}")

    return df

In [ ]:
# Application of the domain specific script to subset existing MESH setups

In [ ]:
# Define paths and directories
directory = r'K:\MESH_model_run'
input_basin = os.path.join(directory, "geofabric", "agg_MERIT_CanTrans_subbasins.shp")
input_river = os.path.join(directory, "geofabric", "agg_MERIT_CanTrans_rivers.shp")
input_ddb_nc = os.path.join(directory, "CanTrans_MESH_drainage_database.nc")
input_parameters_nc = os.path.join(directory, "CanTrans_MESH_parameters.nc")
input_climate_nc = os.path.join(directory, "CanTrans_MESH_forcing.nc")
input_class_ini = os.path.join(directory, "CanTrans_MESH_parameters_CLASS.ini")

### Path to the output files
output_basin = os.path.join(directory, "geofabric", "out_agg_MERIT_CanTrans_subbasins.shp")
output_river = os.path.join(directory, "geofabric", "out_agg_MERIT_CanTrans_rivers.shp")
output_ddb_nc = os.path.join(directory, "out_MESH_drainage_database.nc")
output_parameters_nc = os.path.join(directory, "out_MESH_parameters.nc")
output_climate_nc = os.path.join(directory, "out_MESH_forcing.nc")
output_class_ini = os.path.join(directory, "out_MESH_parameters_CLASS.ini")

In [ ]:
start_comids = [72040181, 72041746, 72040491, 72046739, 72041555, 72040216, 72047734, 72050334, 72047702, 72046658, 72041796, 72047261, 72046809, 72046253, 72047146, 
72047717, 73000830, 72034728, 72034794, 72035842, 72034703, 72035781, 72037904, 72036358, 72040458, 72040296, 72040947, 72044316, 72040957, 72042331, 
72047379, 72050055, 72048858, 72049422, 72043969, 72046864, 72054611, 72048321, 72049269, 72049050, 72047867, 72051729, 72051832, 72052096, 72053913, 
72051848, 72047019, 72047986, 72041533, 72047652, 72050523, 72036265, 72039443, 72036611, 72034667, 72030936, 72030584, 72030851, 72030961, 72035055, 
72032282, 72035053, 72040848, 72041007, 72043410, 72042549, 72040329, 72041562, 72042068, 72041240, 72049458, 72040927, 72041155, 72046841, 72049870, 
72047573, 72048209, 72047226, 72051294, 72053154, 72051928, 72052064, 72051740, 72052534, 72052492, 72052095, 72054315, 72052119, 72052405, 72051937, 
72051968, 72051442, 72052065, 72052462, 72052372, 72052314, 72052131, 72053415, 72053164, 72051402, 72055814, 72052364, 72055915, 72055242, 72055690, 
72057171, 72055476, 72055698, 72055897, 72052797, 72055698, 72058086, 72055914, 72055908, 72056570, 72056000, 72056292, 72056632, 72055898, 72057225, 
72057215, 72055990, 72058563, 72055599, 72055511, 72056032, 72056882, 72055588, 72058531, 72055624, 72057015, 72057219, 72057383, 72055947, 72052775, 
72052773, 72051951, 72053132, 72052299, 72055351, 72053187, 72052141, 72052577, 72052635, 72053843, 72051901, 72051656, 72051969, 72053821, 72052278, 
72035330, 72051361, 72051385, 72047191, 72047474, 72051053, 72047278, 72048064, 72048619, 72052703, 72047350, 72048686, 72048739, 72021217, 72026561, 
72027373, 72027626, 72031106, 72034863, 72035799, 72031222, 72032933, 72030611, 72032343, 72031125, 72034028, 72031828, 72035654, 72031513, 72030415, 
72032697, 72030124, 72032368, 72032058, 72030930, 72039368, 72035170, 72035536, 72038162, 72036257, 72036322, 72035259, 72036733, 72035479, 72042181, 
72045751, 72035601, 72037921, 72038815, 72039094, 72038500, 72041714, 72045518, 72042081, 72037042, 72044349, 72045535, 72043925, 72009146, 72016400, 
71011592, 71023019, 71021952, 71017230, 71011810, 71016791, 71027647, 71022119, 71027932, 71040396, 71040168, 71040195, 71033879, 71027938, 71039779, 
71044935, 71027904, 71041096, 71040972, 71041176, 71040646, 71040771, 71039742, 71043533, 71035768, 71039979, 71040959, 71039751, 71040879, 71040709, 
71037139, 71034011, 71029071, 71028585, 71029391, 71028377, 71028303, 71027787, 71035521, 71035375, 71035188, 71035362, 71036132, 71035810, 71037856, 
71035184, 71037868, 71028727, 71028847, 71028598, 71029380, 71028734, 71022302, 71029434, 71022220, 71022735, 71022780, 71023542, 71023543, 71026036, 
71023173, 71024049, 71023532, 71028555, 71028765, 71022109, 71032592, 71027799, 71028775, 71029446, 71031951, 71034615, 71034828, 71029794, 71021680, 
71023021, 71023704, 71021905, 71023273, 71026598, 71022663, 71022852, 71017098, 71017862, 71017689, 71018145, 71019300, 71017288, 71017347, 71017377, 
71018689, 71016728, 71017186, 71019920, 71017949, 71017623, 71022189, 71016455, 71026674, 71023700, 71022161, 71023222, 71023851, 71025064, 71016944, 
71027819, 71023132, 71027837, 71040418, 71032067, 74000457, 74000322, 74000484, 74000407, 71035343, 71035153, 71028604, 71035048, 71016932, 71017373, 
71022190, 71022254, 71021994, 71023803, 71027420, 71028264, 71035391, 71038679, 71028150, 71041145, 71035121, 71037869, 71028395, 71030139, 71025380, 
71027875, 71023896, 71034318, 71029489, 71035103, 71038842, 71034538, 71034931, 71035453, 71035019, 71039899, 71040491, 71040738, 71040791, 71042823, 
71042034, 71040862, 71034384, 71039798, 71040270, 71041020, 71040325, 71040288, 71040767, 71040330, 71039900, 71039749, 71039991, 71040021, 71040368, 
71039528, 71040961, 71040867, 71040666, 71040642, 71040449, 71040669, 71041483, 71040532, 71040347, 71042263, 71041047, 71040142, 71040708, 71035231, 
71036474, 71040014, 71043885, 71044635, 71040188, 71034266, 71035058, 71035083, 71035242, 71028122, 71028468, 71027467, 71034646, 71027713, 71007741, 
71008012, 71008513, 71007675, 71008741, 71004899, 71004808, 71004835, 71004408, 71012393, 71011800, 71012682, 71011583, 71012485, 71008378, 71007952, 
71004624, 71002488, 71002659, 71002399, 71002639, 71002512, 71001080, 83016083, 82042610, 82042468, 82043065, 82042557, 82040891, 82042994, 82042453, 
82042531, 82042830, 82042669, 82042662, 82042591, 82042639, 82042567, 82040863, 82042465, 82041396, 82041032, 82041246, 82041129, 82042560, 82042507, 
82041268, 82041321, 82041627, 82041165, 82041375, 82042184, 82041848, 82038176, 82038206, 82038771, 82039710, 82037917, 82042216, 82038086, 82038453, 
82039097, 82041177, 82038494, 82041446, 82038832, 82040883, 82038485, 82034626, 82038385, 82038753, 82035029, 82035056, 82035638, 82034802, 82030087, 
82030212, 82034873, 82034982, 82034948, 82038109, 82038100, 82038195, 82038189, 82038087, 82041259, 82038099, 82034814, 82034840, 82037952, 82038020, 
82038166, 82039844, 82038281, 82038379, 82038047, 82034965, 82038349, 82038620, 82038432, 82034905, 82035490, 82035656, 82035078, 82040909, 82040938, 
82041000, 82038475, 82038041, 82039296, 82038990, 82037914, 82041076, 82040994, 82041084, 82038645, 82037984, 82034837, 82034794, 82035821, 82030926, 
82038456, 82035097, 82035042, 82031084, 82030237, 82030290, 82030614, 82030740, 82025654, 82030901, 82026088, 82026186, 82010397, 82017578, 82017660, 
82014117, 81024208, 81029617, 81030977, 81029331, 78000377, 78002147, 78002409, 78003483, 78005579, 78004123, 78004211, 78004092, 78004511, 78005049, 
78004244, 78004360, 78004200, 78004377, 78008613, 78007915, 78006832, 78006958, 78005611, 78004231, 78004395, 78004399, 78005405, 78010437, 78010321, 
78012497, 78013257, 78013257, 78013533, 78010221, 78008860, 78008334, 78014177, 78014522, 78014219, 78015281, 78014375, 78014364, 78013408, 78012544, 
78013425, 78012986, 78012434, 78010574, 78010151, 78011538, 78003932, 78003926, 78003199, 78004113, 78005682, 78005223, 78005288, 78005699, 78004049, 
78004791, 78004184, 78004077, 78006845, 78008294, 78006955, 78007081, 78007052, 78008666, 78008938, 78007293, 78009016, 78010891, 78010204, 78010168, 
78008702, 78011790, 78011600, 78009026, 78010285, 78013579, 78012423, 78011527, 78006775, 78008526, 78008535, 78009681, 78008690, 78009003, 78011699, 
78010198, 78010640, 78010487, 78013314, 78012179, 78012369, 78012478, 78010402, 78012098, 78012048, 78013016, 78012243, 78012250, 78012933, 78012065, 
78013209, 78012422, 78008413, 78008653, 78008403, 78008760, 78008835, 78009121, 78008705, 78007214, 78008661, 78010220, 78010542, 78010736, 78014077, 
78012204, 78013192, 78012225, 78012940, 78010269, 78013440, 78012688, 78012215, 78010424, 78012949, 78012970, 78012354, 78010248, 78013476, 78011919, 
78013340, 78013323, 78011964, 78012737, 78012172, 78012166, 78012310, 78012155, 78012473, 78012228, 78012217, 78012536, 78013427, 78012642, 78011113, 
78013328, 78013328, 78013328, 78012303, 78012563, 78012661, 78012553, 78005565, 78006141, 78005939, 81030128, 81034197, 81029358, 81030415, 81034085, 
81029676, 81024148, 81018832, 81024059, 81024048, 81019656, 81024318, 81024473, 81024537, 81015017, 81019068, 81014678, 81015189, 81014516, 81006891, 
82021235, 82021308, 82026385, 82025519, 82026297, 82026950, 82025345, 82030215, 82034857, 82026984, 82031408, 82031914, 82017297, 82017565, 82021780, 
82018635, 82021477, 82017526, 82017347, 82017732, 82008577, 82006932, 82002296, 82002281, 82007001, 82010650, 82000289, 83006171, 83002962, 84006587, 
85009406, 74000271, 74001720, 74000430, 74000315, 72040863, 73000453, 73000663, 73000849, 73000783, 73000925, 73001025, 73001843, 73001771, 73002241, 
73002067, 73002442, 73003733, 73004772, 72041760, 72041048, 72040349, 72041505, 72042638, 72040114, 72046127, 72046585, 72042504, 72046928, 72047270, 
72047028, 72059423, 72057217, 72055577, 72051286, 72051520, 72046420, 72047462, 72056630, 72058177, 72059517, 72059542, 72059753, 72058362, 72058365, 
72055995, 72058329, 72055236, 72058582, 72055352, 72051993, 71047102, 71047086, 71047232, 71047203, 71044653, 71044005, 71044206, 71044075, 71044056, 
74010191, 74010867, 74003342, 78014338, 78015092, 78014244, 78014127, 78013776, 78017191, 78016861, 78016721, 78016508, 78016043, 78016628, 78016011, 
78015712, 78015603, 78015352, 78014191, 78018410, 78024332, 78024601, 78025711, 78024341, 78027334, 78028215, 78021942, 78022737, 78020032, 78022315, 
78020203, 78017893, 78017742, 78017906, 78020347, 78023764, 78021828, 78020238, 78021157, 78020663, 78020663, 78021143, 78021115, 78024741, 78022530, 
78023280, 78022488, 78022486, 78022389, 78022457, 78022543, 78018257, 78020358, 78018213]

In [ ]:
# Get all upstream subbasins from multiple given outlets (start ranks)
# Subset: basin and river
gdf_basin, gdf_river = subset_upstream_basins_shapefile(
    input_basin_path=input_basin,
    input_river_path=input_river,
    start_ranks=start_comids,
    rank_id_col='COMID',
    next_id_col='NextDownID',
    unit_area_col='unitarea',
    upstream_area_col='uparea',
    output_basin_path=output_basin,
    output_river_path=output_river,
    include_downstream=True  # Optional
)

In [ ]:
# Subset MESH_drainage_database.nc for the domain of interest
output_nc, maximum_rank = subset_netcdf_ddb(
    output_basin,
    input_ddb_nc,
    output_ddb_nc
)

In [ ]:
# Subset MESH_parameters.nc for the domain of interest
output_nc_params = subset_netcdf_climate_and_parameters(
    output_ddb_nc,
    input_parameters_nc,
    output_parameters_nc,
    Key_column='subbasin',
    lat_name='lat',
    lon_name='lon'
)

In [ ]:
# Subset MESH_forcing.nc for the domain of interest
output_nc_climate = subset_netcdf_climate_and_parameters(
    input_ddb = output_ddb_nc,
    input_nc=input_climate_nc,
    output_nc=output_climate_nc,
    Key_column='ID',
    lat_name='lat',
    lon_name='lon'
)

In [ ]:
# Update the required MESH input file (MESH_parameters_CLASS.ini)
maximum_rank = len(xr.open_dataset(output_ddb)['Rank'].values.astype(int))
edit_TotalNumOfGridsSubbasins_value(input_class_ini, output_class_ini, maximum_rank)

In [ ]:
# Extract time series data from netcdf file at location of interest
# Option 1-CaSRv2p1: extract by index
nc_forcing_path = r'K:\NAS_Project\NAS_Model_Runs\MESH_Run_0p0\MESH_input_CanTrans_CaSRv2p1_1980_2018.nc'
dim = 'subbasin'
variable = 'RDRS_v2.1_P_TT_09944'
target_subbasin = 47576 #4665 #5676
df = extract_timeseries_by_subbasin(nc_forcing_path, subbasin_dim=dim, variable_name=variable, target_subbasin=target_subbasin, save_csv=True)
print(df.head())

In [ ]:
# Extract time series data from netcdf file at location of interest
# Option 1-CaSRv3p1: extract by index
root_dir = r'R:\NAS_CaSR'
nc_forcing_path = os.path.join(root_dir, "CanTrans_CaSRv2p1_MESH_forcing.nc")
dim='x'
variable = 'RDRS_v2.1_P_TT_09944'
target_subbasin = 44629 #4665 #5676
df = extract_timeseries_by_subbasin(nc_forcing_path, subbasin_dim=dim, variable_name=variable, target_subbasin=target_subbasin, save_csv=True)
print(df.head())

# Option 2: extract by nearest lat/lon
#df = extract_timeseries_by_subbasin(nc_forcing_path, variable_name=variable, target_latlon=(106.3, 56.2), save_csv=True)
#print(df.head())

In [2]:
import xarray as xr
import numpy as np
import os

# Load dataset
ds = xr.open_dataset("MESHflow_out/MESH_drainage_database_Polish_0p05_0p02_0p01.nc")

# Output directory
output_dir = "MESHflow_out"
os.makedirs(output_dir, exist_ok=True)

# Chunking details
total_size = ds.sizes['subbasin']
chunk_size = 512
num_chunks = (total_size + chunk_size - 1) // chunk_size

# Loop through chunks
for i in range(num_chunks):
    start = i * chunk_size
    end = min(start + chunk_size, total_size)
    
    chunk = ds.isel(subbasin=slice(start, end))
    
    # Rename existing Rank and Next if they exist
    if 'Rank' in chunk:
        chunk = chunk.rename({'Rank': 'oldRank'})
    if 'Next' in chunk:
        chunk = chunk.rename({'Next': 'oldNext'})
    
    # Reset Rank to start at 1 within each chunk
    local_size = end - start
    new_rank = xr.DataArray(np.arange(1, local_size + 1), dims=["subbasin"])
    chunk['Rank'] = new_rank

    # Create new Next: increment by 1, last item is 0
    next_values = np.arange(2, local_size + 2)
    next_values[-1] = 0
    new_next = xr.DataArray(next_values, dims=["subbasin"])
    chunk['Next'] = new_next

    # Save chunk to file
    filename = f"MESH_drainage_database_{start}_to_{end-1}.nc"
    filepath = os.path.join(output_dir, filename)
    chunk.to_netcdf(filepath)

In [34]:
import pandas as pd
import glob
import os

def generate_mean_summary(input_folder, output_file):
    """
    For each CSV file and zone_id, get mean values as formatted strings,
    then pivot so zone_ids become columns, filenames rows.

    Args:
        input_folder (str): Folder with CSV files containing 'zone_id', 'Layer', 'mean'
        output_file (str): Output CSV path
    """
    csv_files = glob.glob(os.path.join(input_folder, '*.csv'))

    if not csv_files:
        raise ValueError("No CSV files found in the specified folder")

    records = []

    for file in csv_files:
        filename = os.path.splitext(os.path.basename(file))[0]
        df = pd.read_csv(file, usecols=['zone_id', 'Layer', 'mean'])
        df['mean'] = df['mean'].round(2)

        for zone_id, group in df.groupby('zone_id'):
            group = group.sort_values(by='Layer')
            means = group['mean'].tolist()
            # means_str = '{' + ','.join(f"{m:.2f}" for m in means) + '}'
            means_str = '{' + ','.join(f"{int(round(m))}" for m in means) + '}'
            records.append({
                'zone_id': int(zone_id),
                'filename': filename,
                'Layers': means_str
            })

    result_df = pd.DataFrame(records)

    # Pivot so zone_id columns, filename rows
    pivot_df = result_df.pivot(index='filename', columns='zone_id', values='Layers')

    # Optional: rename zone_id columns to strings
    pivot_df.columns = [str(col) for col in pivot_df.columns]

    # Reset index to have filename as a column
    pivot_df = pivot_df.reset_index()

    # Save to CSV
    pivot_df.to_csv(output_file, index=False)
    print(f"Pivoted mean summary saved to: {output_file}")
    return pivot_df

In [35]:
# Example usage
input_folder = 'CanTransStatOut/soil'  # Folder containing input CSV files
output_file = 'CanTransStatOut/soil/zone_mean_summary.csv'
df = generate_mean_summary(input_folder, output_file)
print(df.head())

Pivoted mean summary saved to: CanTransStatOut/soil/zone_mean_summary.csv
  filename              0              1              2              5  \
0     CLAY  {13,12,12,11}  {14,14,15,14}  {16,16,15,16}  {14,15,16,16}   
1       OC      {8,4,2,0}    {15,11,9,2}   {22,19,15,2}    {16,11,7,2}   
2     SAND  {52,52,54,56}  {52,55,57,54}  {48,50,51,51}  {51,53,55,52}   

               6              8             10             11             12  \
0  {14,15,17,15}  {14,15,17,16}  {16,18,18,16}  {12,11,12,13}  {13,11,12,13}   
1    {18,10,7,2}    {17,11,7,1}     {11,8,6,2}      {6,4,2,0}      {7,4,3,0}   
2  {52,56,57,54}  {47,48,51,49}  {50,52,54,51}  {56,55,55,53}  {55,54,54,53}   

              13             14             15             16             17  \
0  {12,10,12,12}  {18,19,20,19}  {22,25,27,23}  {12,12,14,15}  {18,21,23,20}   
1      {5,2,1,0}   {35,30,24,6}      {7,4,2,1}      {5,2,1,0}      {9,6,3,2}   
2  {57,55,57,58}  {42,44,46,46}  {39,38,39,36}  {57,56,53,52}  {44,4